In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile
from urllib import request
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import tensorflow as tf
import cv2

In [ ]:
img_database = np.loadtxt('/kaggle/input/fashionmnist/fashion-mnist_train.csv', delimiter= ',', skiprows = 1)[:, 1:]
print(img_database.shape)

There are 60000 images and the dimension is 784,1. It basically means that the images has been flattened out to an one dimensional array of 784,1 from the actual dimension of 28X28

In [ ]:
total_num_images = (img_database.shape[0])

**The Neural Network Archi**

In [ ]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

tensorflow.placeholder was not working so found above solution in StackOverFlow. :)|

In [ ]:
#structuring the input and output layers
n_input = 784
hidden_layer1 = 256
hidden_layer2 = 32
hidden_layer3 = 32
hidden_layer4 = 256
output_layer = 784 #this is mainly encoding and decoding

learning_rate = 0.1
epochs = 150
batch_size = 100


#placeholders
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, output_layer])

#weight and biases
Weight_NN = {"W1": tf.Variable(tf.random_normal([n_input, hidden_layer1])),
             "W2": tf.Variable(tf.random_normal([hidden_layer1, hidden_layer2])),
            "W3": tf.Variable(tf.random_normal([hidden_layer2, hidden_layer3])),
            "W4": tf.Variable(tf.random_normal([hidden_layer3, hidden_layer4])),
            "W5": tf.Variable(tf.random_normal([hidden_layer4, output_layer]))
            }
Bias_NN = { "B1": tf.Variable(tf.random_normal([hidden_layer1])),
           "B2": tf.Variable(tf.random_normal([hidden_layer2])),
          "B3": tf.Variable(tf.random_normal([hidden_layer3])),
          "B4": tf.Variable(tf.random_normal([hidden_layer4])),
          "B5": tf.Variable(tf.random_normal([output_layer]))}

*Point to be noted*: The variable constructor for weights and biases are used because they will be changing.

* **Placeholders:** A placeholder is simply a variable that we will assign data to at a later date. It allows us to create our operations and build our computation graph, without needing the data.
* **The Variable()** constructor requires an initial value for the variable, which can be a Tensor of any type and shape. This initial value defines the type and shape of the variable. After construction, the type and shape of the variable are fixed.


In [ ]:
#forward pass for the neural network
Z1 = tf.add(tf.matmul(X, Weight_NN["W1"]), Bias_NN["B1"])
Z1_out = tf.nn.sigmoid(Z1) #non linearity applied

Z2 = tf.add(tf.matmul(Z1_out, Weight_NN["W2"]), Bias_NN["B2"])
Z2_out = tf.nn.sigmoid(Z2)

Z3 = tf.add(tf.matmul(Z2_out, Weight_NN["W3"]), Bias_NN["B3"])
Z3_out = tf.nn.sigmoid(Z3)

Z4 = tf.add(tf.matmul(Z3_out, Weight_NN["W4"]), Bias_NN["B4"])
Z4_out = tf.nn.sigmoid(Z4)

Z5 = tf.add(tf.matmul(Z4_out, Weight_NN["W5"]), Bias_NN["B5"])
NN_output = (Z5)

In [ ]:
#the same NN using tensorflow api
#Z1 = tf.layers.dense(X, hidden_layer1, activation = tf.nn.sigmoid)
#Z2 = tf.layers.dense(Z1, hidden_layer2, activation = tf.nn.sigmoid)
#Z3 = tf.layers.dense(Z2, hidden_layer3, activation = tf.nn.sigmoid)
#Z4 = tf.layers.dense(Z3, hidden_layer4, activation = tf.nn.sigmoid)
#NN_output = tf.layers.dense(Z4, output_layer)

**Dataset Processing**

In [ ]:
np.random.shuffle(img_database) #shuffling
X_train = img_database
X_train = X_train #normalize
X_train_noisy = X_train + 10*np.random.normal(0, 1, size = X_train.shape) #creating noisy dataset

#original img
plt.imshow(X_train[0].reshape(28,28), cmap = 'gray')
plt.show()

#noisy img
plt.imshow(X_train_noisy[0].reshape(28,28), cmap = 'gray')
plt.show()

In [ ]:
#loss function
computed_loss = tf.reduce_mean(tf.square(NN_output-Y))
optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(computed_loss)
init = tf.global_variables_initializer() #these are the wieghts and biases

*This optimizer in neural network works as follows:*

* It begins with a few coefficients, observes their cost, and finds a cost value lower than what it is currently.
* It proceeds towards the lower weight and updates the coefficients’ value.
* The process iterates until the local minimum is attained. A local minimum is a point outside which it can’t proceed.

**Execution of the Neural Network**

In [ ]:
ses = tf.Session()
ses.run(init)
for epoch in range(epochs):
    for i in range(int(total_num_images/batch_size)):
        X_epoch = X_train[i*batch_size : (i+1)*batch_size]
        X_noise_epoch = X_train_noisy[i*batch_size : (i+1)*batch_size ]
        _, loss = ses.run([optimizer, computed_loss], feed_dict={X: X_noise_epoch, Y: X_epoch})
    print('Epoch', epoch, '/', epochs, 'loss: ', loss)

#choosing an img
X_actual = X_train[20:30]
noisy_image = X_train_noisy[20:30]

denoised = ses.run(NN_output, feed_dict = {X:noisy_image})

#printing
fig, axes = plt.subplots(nrows = 3, ncols = 10, sharex = True, sharey= True, figsize=(20,4))
for images, row in zip([X_actual, noisy_image, denoised], axes):
    for img, ax in zip(images, row):
        ax.imshow(img.reshape((28,28)), cmap = 'Greys_r')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        

fig.tight_layout(pad = 0.1)

The X is mapped to X_noise_epoch because here what we are doing at the beginning of the neural network is that, we are making the original images noisy first. Then the output Y is mapped to the image of X_epochs which are the denoised version.

**This is just an implementation using neural networks. Using CNN can further improve the denoising**